In [1]:
import math
from textblob import TextBlob as tb
import newspaper
import os
import random
import pickle
from datetime import datetime
from tqdm import tqdm as tq

In [2]:
import import_ipynb
from prepare import collector
from lexrank_summarizer import lexrank
from news_classes import customizedPaper
from news_classes import paragraph

importing Jupyter notebook from prepare.ipynb
importing Jupyter notebook from news_classes.ipynb
importing Jupyter notebook from lexrank_summarizer.ipynb
[1.         0.16068673 0.125      0.10715664 0.125      0.125
 0.125      0.125     ]
F


In [3]:
from werkzeug.wrappers import Request, Response
from flask import Flask
from flask import render_template
from flask import request

In [4]:
import _thread
import time

from werkzeug.wrappers import Request, Response
from flask import Flask
from flask import render_template
from flask import request

In [5]:
cwd=os.getcwd()+'/'
news_path='news_today.pkl'
summarizer=lexrank(.1,.0001)

In [6]:
with open(news_path, 'rb') as input:
    allnews = pickle.load(input)

In [7]:
def worker():
    while True:
        collector.prepare_news()
        print("News Collected")
        collector.prepare_idf()
        print("Idf Stored")
        with open(news_path, 'rb') as input:
            allnews = pickle.load(input)

In [8]:
'''
try:
   _thread.start_new_thread(worker,())
except:
   print ("Error: unable to start thread")
'''

'\ntry:\n   _thread.start_new_thread(worker,())\nexcept:\n   print ("Error: unable to start thread")\n'

In [9]:
def getSummarizedFromList(para_list):
    summ_list=[]
    for paragraph in para_list:
        obj={'title':paragraph.title, 'url':paragraph.url, 'text':summarizer.summarize(paragraph.body), 'date':paragraph.date, 'brand':paragraph.brand}
        summ_list.append(obj)    
        
    return summ_list

In [10]:
def getMultiDocSummaryFromList(para_list):
    blob_news=""
    for paragraph in para_list:
        blob_news+=paragraph.body
    
    return summarize(blob_news, 5)

In [11]:
def searchNews(searched_key, min_count):
    key_words=searched_key.split()
 
    para_list=[]
    
    for paragraph in allnews.paragraphs:
        word_matched=0
        letter_matched=0
        for word in key_words:
            word=word.lower()
            if paragraph.title is not None and paragraph.body is not None:
                if word in paragraph.title.lower().split():
                    word_matched+=1
                    letter_matched+=len(word)
        if word_matched>=min_count and paragraph.date is not None:
            paragraph.setCounts(word_matched)
            paragraph.setLetterMatched(letter_matched)
            para_list.append(paragraph)
    return para_list

In [12]:
def sortNews(para_list):
    para_list.sort(key=lambda item:(item.cnt, item.ltr, item.date.year, item.date.month, item.date.day), reverse=True)
    
    return para_list

In [13]:
list_of_news=[]
searched_word=""
selected_item=""
total_page=0
content_per_page=5

In [14]:
app = Flask(__name__)

In [15]:
def singleNewsFromKey(searched_key):
    para_list = searchNews(searched_key, 1)
    para_list = sortNews(para_list)

    return para_list

In [16]:
def multiNewsFromKey(searched_key):
    para_list = searchNews(searched_key, len(searched_key.split()))
    para_list = sortNews(para_list)

    return para_list

In [17]:
@app.route('/show_result', methods=['GET', 'POST'])
def show_result():
    global searched_word
    searched_word = request.form['news_keyword']
    global selected_item
    selected_item = request.form.get('item')
    
    global list_of_news
    
    
    if selected_item == "single":
        list_of_news=singleNewsFromKey(searched_word)

        global total_page
        total_page=math.ceil(len(list_of_news)/content_per_page)

        list_ten=[]
        for i in range(min(len(list_of_news), content_per_page)):
            list_ten.append(list_of_news[i])
            
        rendered = render_template('ani_show_news.html', news_list=getSummarizedFromList(list_ten), number_of_pages=total_page, keyword=searched_word)
    else:
        list_of_news=multiNewsFromKey(searched_word)

        list_ten=[]
        for i in range(min(len(list_of_news), content_per_page)):
            list_ten.append(list_of_news[i])
            
        rendered = render_template('ani_show_multi_news.html', multi_news=getMultiDocSummaryFromList(list_ten))
    
    return rendered

In [18]:
@app.route('/show_other_page', methods=['POST'])
def show_other_page():
    page_number = int(request.form['page_number'])
        
    list_ten=[]
    for i in range(page_number*content_per_page, min(len(list_of_news), (page_number+1)*content_per_page)):
        list_ten.append(list_of_news[i])
    
    rendered = render_template('ani_show_news.html', news_list=getSummarizedFromList(list_ten), number_of_pages=total_page, keyword=searched_word)

    return rendered

In [19]:
@app.route("/")
def search_key():
    rendered = render_template('ani_search.html')
    return rendered

In [20]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('0.0.0.0', 9001, app)

 * Running on http://0.0.0.0:9001/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Dec/2018 18:47:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2018 18:47:52] "POST /show_result HTTP/1.1" 200 -


[1.         0.10714056 0.125      0.125      0.10714056 0.125
 0.125      0.125     ]
W
[1.         0.1        0.1        0.0882524  0.1        0.08821698
 0.1        0.11763911 0.1        0.0882524 ]
W
[1.         0.25       0.32142704 0.21428648]
W
[1.         0.06302137 0.06290875 0.07880005 0.03153223 0.07871809
 0.06300698 0.09440649 0.07877476 0.04718107 0.07861718 0.06289158
 0.09436162 0.09435127]
"
[1.         0.13227879 0.11111111 0.11111111 0.11111111 0.10580578
 0.10580578 0.13227879 0.11111111]
W
